# sCellST Benchmark (INT25-INT28)

## Environment
It is recommended to use a separate environment for sCellST.

```bash
conda create -n scellst_bench python=3.10
conda activate scellst_bench

# Install PyTorch (CUDA 11.8)
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
pip install scanpy pandas numpy matplotlib seaborn scprep tqdm pytorch-lightning omegaconf loguru anndata scikit-learn dask[dataframe]

# Install sCellST dependencies if not covered above. Usually sCellST requires specific versions.
# Assuming sCellST source code is in benchmark/sCellST

# === IMPORTANT: OpenCV Fix ===
# If you encounter "DLL load failed" for cv2, try installing headless version:
pip uninstall opencv-python -y
pip install opencv-python-headless
```

In [1]:
import os
import sys
from pathlib import Path

# === Windows DLL Fix ===
# Fix for OSError: [WinError 1114] Error loading c10.dll / libiomp5md.dll etc.
if os.name == 'nt':
    conda_prefix = os.environ.get('CONDA_PREFIX')
    possible_paths = []
    if conda_prefix:
        possible_paths.append(Path(conda_prefix) / 'Library' / 'bin')
    
    # Add current python dir's Library/bin if distinct
    curr_python = sys.executable
    if curr_python:
        possible_paths.append(Path(curr_python).parent / 'Library' / 'bin')

    for p in possible_paths:
        if p.exists():
            print(f"Adding DLL path: {p}")
            os.environ['PATH'] = str(p) + os.pathsep + os.environ['PATH']
            try:
                os.add_dll_directory(str(p))
            except AttributeError:
                pass 

import pandas as pd
import numpy as np
import torch
import json
import shutil

# Set Paths
ROOT = Path(os.environ.get('MORPHO_VC_ROOT', '../')).expanduser().resolve()
BENCHMARK_DIR = ROOT / 'benchmark'
SCELLST_DIR = BENCHMARK_DIR / 'sCellST'
RESULT_DIR = BENCHMARK_DIR / 'results' / 'sCellST'
DATA_DIR = ROOT / 'data'
HEST_DIR = DATA_DIR / 'hest_data'

# Add paths to sys.path
# 1. sCellST
sys.path.append(str(SCELLST_DIR))

# 2. HEST (Local third_party)
HEST_SRC = ROOT / 'third_party' / 'HEST' / 'src'
if HEST_SRC.exists():
    print(f"Adding HEST local source: {HEST_SRC}")
    sys.path.insert(0, str(HEST_SRC))
else:
    print("Warning: local HEST source not found in third_party. If 'hest' is not installed via pip, this will fail.")

RESULT_DIR.mkdir(parents=True, exist_ok=True)

print(f"ROOT: {ROOT}")
print(f"HEST_DIR: {HEST_DIR}")
print(f"SCELLST_DIR: {SCELLST_DIR}")

# sCellST imports
# Fix for dask-expr RuntimeError in spatialdata
try:
    import dask
    dask.config.set({'dataframe.query-planning': False})
except ImportError:
    pass

try:
    from scellst.submit_function import embed_cells
    from scellst.dataset.data_module import STDataModule, MilVisiumHandler
    from scellst.utils.io_utils import load_config
    from scellst.utils.utils import prepare_model
    from scellst.trainer import prepare_trainer
    from omegaconf import OmegaConf, DictConfig
    from lightning.pytorch import seed_everything
    import lightning as L
    import scellst.type
    import scellst.lightning_model.gene_lightning_model
except ImportError as e:
    print("\n" + "="*60)
    print("IMPORT ERROR DETECTED")
    print(f"Error: {e}")
    if 'cv2' in str(e):
        print("\nSUGGESTION: This looks like an OpenCV DLL error.")
        print("Try running the following in your terminal:")
        print("  pip uninstall opencv-python -y")
        print("  pip install opencv-python-headless")
    elif 'loguru' in str(e):
        print("\nSUGGESTION: Missing 'loguru'. Install it via:")
        print("  pip install loguru")
    print("="*60 + "\n")
    raise e



from enum import Enum

# --- Fix 1: Runtime Monkeypatch for TaskType ---
# Fixes 'AttributeError: att_regression' by adding missing enum members at runtime
class PatchedTaskType(str, Enum):
    regression = "regression"
    nb_total_regression = "nb_total_regression"
    nb_mean_regression = "nb_mean_regression"
    # The missing members that cause the crash:
    bag_regression = "bag_regression"
    att_regression = "att_regression"

    @classmethod
    def list(cls):
        return list(map(lambda c: c.value, cls))

# Apply the patch
scellst.type.TaskType = PatchedTaskType

scellst.lightning_model.gene_lightning_model.TaskType = PatchedTaskType
print("Success: Monkeypatched TaskType to fix library inconsistencies.")

Adding DLL path: C:\ProgramData\anaconda3\envs\scellst_bench\Library\bin
Adding DLL path: C:\ProgramData\anaconda3\envs\scellst_bench\Library\bin
Adding HEST local source: D:\code\Morpho-VC\third_party\HEST\src
ROOT: D:\code\Morpho-VC
HEST_DIR: D:\code\Morpho-VC\data\hest_data
SCELLST_DIR: D:\code\Morpho-VC\benchmark\sCellST


C:\ProgramData\anaconda3\envs\scellst_bench\lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
C:\ProgramData\anaconda3\envs\scellst_bench\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


19:20:41 | INFO | PROJ_ROOT path is: D:\code\Morpho-VC\benchmark\sCellST


C:\ProgramData\anaconda3\envs\scellst_bench\lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


Success: Monkeypatched TaskType to fix library inconsistencies.


C:\ProgramData\anaconda3\envs\scellst_bench\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


## 1. Prepare Data & Embeddings

Load common genes and generate cell embeddings using sCellST's Pre-trained ResNet50 (ImageNet).

In [2]:
# Load Common Genes
common_gene_path = DATA_DIR / 'spatial_data' / 'common_genes.txt'
with open(common_gene_path, 'r') as f:
    common_genes = [line.strip() for line in f.readlines()]
print(f"Loaded {len(common_genes)} common genes.")

# Slide Splits
train_ids = ['INT25', 'INT26']
val_ids = ['INT27']
test_ids = ['INT28']
all_ids = train_ids + val_ids + test_ids

# Generate Embeddings (if not exist)
# This step generates .h5 files in data/hest_data/cell_embeddings
# sCellST expects this structure.

TAG = "imagenet-rn50"
MODEL_NAME = "resnet50"
NORM_TYPE = "train"
SHAPE_NAME = "cellvit"

# Check if embeddings exist to avoid re-running
missing_emb = []
for sid in all_ids:
    emb_path = HEST_DIR / "cell_embeddings" / f"{TAG}_{NORM_TYPE}_{sid}_{SHAPE_NAME}.h5"
    if not emb_path.exists():
        missing_emb.append(sid)

if missing_emb:
    print(f"Generating embeddings for {missing_emb}...")
    
    # 1. Convert to sCellST format (generates cell_images/*.h5)
    # This step is required before embed_cells
    print("Step 1: Converting HEST data to sCellST format...")
    try:
        from scellst.cellhest_adapter.processing_utils import convert_to_cellst, filter_data
        
        # We need to pass technology list. Assuming Visium for these INT datasets.
        # Ideally we get this from HEST metadata but let's be explicit if we know.
        # Or we can query HEST metadata dataframe as sCellST does.
        df = pd.read_csv(ROOT / "third_party/HEST/assets/HEST_v1_1_0.csv")
        df = df.set_index("id")
        tech_list = []
        for sid in missing_emb:
            if sid in df.index:
                tech_list.append(df.loc[sid, "st_technology"])
            else:
                 # Fallback default
                 print(f"Warning: {sid} not found in HEST metadata csv, assuming Visium.")
                 tech_list.append("Visium")
        
        convert_to_cellst(
            path_dataset=HEST_DIR,
            ids_to_query=missing_emb,
            technology=tech_list,
            shape_name=SHAPE_NAME
        )
    except Exception as e:
        print(f"Error in convert_to_cellst: {e}")
        import traceback
        traceback.print_exc()

    # 2. Embed Cells
    print("Step 2: Generating Embeddings...")
    try:
        embed_cells(
            path_dataset=HEST_DIR,
            organ=None,
            ids_to_query=missing_emb,
            tag=TAG,
            model_name=MODEL_NAME,
            normalisation_type=NORM_TYPE,
            shape_name=SHAPE_NAME
        )
    except Exception as e:
        # Detailed error printing
        import traceback
        traceback.print_exc()
        print(f"Error generating embeddings: {e}")
        print("\nPotential Issues:")
        print("1. Missing source images/h5ad in hest_data?")
        print(f"   Checking {HEST_DIR} content:")
        if HEST_DIR.exists():
            print(f"   - {len(list(HEST_DIR.glob('*')))} files/dirs found.")
        else:
            print(f"   - {HEST_DIR} NOT FOUND!")
else:
    print("All embeddings found. Skipping generation.")

Loaded 17512 common genes.
All embeddings found. Skipping generation.


## 2. Training

Train using `INT25`, `INT26`. Validate on `INT27`.

In [3]:
# Load Default Config
config_path = SCELLST_DIR / "config" / "gene_default.yaml"
config = load_config(config_path)

# Override Config for Training - Following OFFICIAL sCellST settings
config.data.data_dir = str(HEST_DIR)
# Official approach: Include all training slides in list_training_ids
# The DataModule will automatically split them 80/20 for train/val
config.data.list_training_ids = train_ids + val_ids  # [INT25, INT26, INT27]
config.data.genes = common_genes
config.data.embedding_tag = f"{TAG}_{NORM_TYPE}"
config.data.batch_size = 64  # Official uses 128, reduced for GPU memory
config.data.frac_train = 0.8  # Official default - auto split
config.data.dataset_handler = "mil"
# task_type: Use 'regression' for MSE loss, or 'nb_mean_regression' for NB distribution
# Official model.yaml uses 'regression'
config.model.task_type = "regression"

# Setup output dir
save_dir = RESULT_DIR / "checkpoints"
save_dir.mkdir(parents=True, exist_ok=True)
best_model_path = save_dir / "best_model.ckpt"

seed_everything(config.data.seed)

# Single DataModule with internal train/val split (OFFICIAL approach)
print("Setting up DataModule (Train + Val internal split)...")
from scellst.dataset.data_module import STDataModule
from scellst.dataset.data_handler import MilVisiumHandler

data_dm = STDataModule(
    data_dir=Path(config.data.data_dir),
    list_training_ids=config.data.list_training_ids,
    genes=config.data.genes,
    embedding_tag=config.data.embedding_tag,
    dataset_handler=MilVisiumHandler(),
    batch_size=config.data.batch_size,
    frac_train=config.data.frac_train,  # 0.8 - auto 80/20 split
    fold=-1,  # Use all slides
    predict_id=None,
    num_workers=0,  # Windows compatibility
)
data_dm.prepare_data()
data_dm.setup(stage="fit")

print(f"Train samples: {len(data_dm.train_dataset)}, Val samples: {len(data_dm.val_dataset)}")

# Model
config.model.gene_names = data_dm.get_gene_names()
config.predictor.output_dim = len(config.model.gene_names)
model = prepare_model(config)

# Check if existing checkpoint exists - LOAD instead of retrain
if best_model_path.exists():
    print(f"Found existing checkpoint at {best_model_path}")
    print("Loading pre-trained model (to retrain, delete the checkpoint file first)...")
    # Load weights from checkpoint
    model = type(model).load_from_checkpoint(best_model_path, weights_only=False)
    ckpt_path = str(best_model_path)
    print(f"Model loaded successfully from: {ckpt_path}")
else:
    print("No existing checkpoint found. Starting training...")
    
    # Trainer with Progress Bar (NO TensorBoard/WandB)
    from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, RichProgressBar
    from lightning.pytorch.loggers import CSVLogger
    import lightning.pytorch.callbacks as pl_callbacks
    
    checkpoint_callback = ModelCheckpoint(
        dirpath=save_dir,
        filename='best_model',
        save_top_k=1,
        monitor='val_loss',
        mode='min'
    )
    
    # Official settings: patience=20, max_epochs=400
    es_callback = EarlyStopping(monitor='val_loss', patience=20, mode='min')
    
    # Custom callback: Print one line per epoch for user feedback
    class SimpleEpochProgress(pl_callbacks.Callback):
        def on_train_epoch_end(self, trainer, pl_module):
            metrics = trainer.callback_metrics
            loss = metrics.get("val_loss", float('nan'))
            train_loss = metrics.get("train_loss", float('nan'))
            # Only print if val_loss is available (i.e., after validation runs)
            if not isinstance(loss, float) or loss != float('nan'):
                print(f"Epoch {trainer.current_epoch}: train_loss={float(train_loss):.4f}, val_loss={float(loss):.4f}")
    
    trainer = L.Trainer(
        max_epochs=400,  # Official default
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        callbacks=[
            checkpoint_callback,
            es_callback,
            RichProgressBar(),
            SimpleEpochProgress()
        ],
        default_root_dir=save_dir,
        enable_progress_bar=True,
        logger=CSVLogger(save_dir),
        log_every_n_steps=10,
    )
    
    print("Training with OFFICIAL settings: max_epochs=400, patience=20...")
    # Train using OFFICIAL approach: train_dataloader + val_dataloader from SAME DataModule
    trainer.fit(
        model,
        train_dataloaders=data_dm.train_dataloader(),
        val_dataloaders=data_dm.val_dataloader()
    )
    ckpt_path = checkpoint_callback.best_model_path
    print(f"Training finished. Best model saved to: {ckpt_path}")

Seed set to 0


Setting up DataModule (Train + Val internal split)...
19:20:49 | INFO | Preparing data for training.
19:20:49 | INFO | Preparing data for ID: INT25.
19:20:49 | INFO | Loading data for ID: INT25
19:20:49 | INFO | Loaded adata with shape: (3808, 17943)
19:20:49 | INFO | First obs names: Index(['AAACAAGTATCTCCCA-1_INT25', 'AAACACCAATAACTGC-1_INT25',
       'AAACAGAGCGACTCCT-1_INT25', 'AAACAGGGTCTATATT-1_INT25',
       'AAACATTTCCCGGATT-1_INT25'],
      dtype='object')
19:20:49 | INFO | After genes filtering: (3808, 9795)
19:20:49 | INFO | After count filtering: (3804, 9795)
19:20:49 | INFO | After mt and rps filtering: 9795
19:20:49 | INFO | Normalising spot counts.
19:20:49 | INFO | Log1p transform counts.
19:20:50 | INFO | Found 3440 / 3804 spots in the adata file.
19:20:50 | INFO | Found 3440 / 3440 spots in the cell embedding file.
19:20:50 | INFO | Preprocessing completed.
19:20:50 | INFO | Selecting genes from list.
19:20:50 | INFO | Found 9586 / 17512 genes in adata.var_names
19:20

C:\ProgramData\anaconda3\envs\scellst_bench\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


19:20:56 | INFO | GeneLightningModel(
  (train_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef(),
    prefix=val/
  )
  (valid_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef(),
    prefix=val/
  )
  (test_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef()
    (scc): SpearmanCorrCoef()
  )
  (model): InstanceMilModel(
    (gene_predictor): GenePredictor(
      (final_activation_layer): Softplus(beta=20, threshold=20.0)
      (model): Sequential(
        (0): Sequential(
          (0): Linear(in_features=2048, out_features=256, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
          (2): Dropout(p=0.1, inplace=False)
        )
        (1): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
          (2): Dropout(p=0.1, inplace=False)
        )
        (2): Sequential(
          (0): Linear(in_f

## 3. Prediction (INT28)

Load best model and predict on `INT28`.

In [4]:
# --- Manual Prediction Setup ---
print(f"Loading model from {ckpt_path}")
# FIX: Set weights_only=False to fix UnpicklingError with DictConfig
model = type(model).load_from_checkpoint(ckpt_path, weights_only=False)
model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Retrieve the exact gene list used during training (from model hparams)
train_gene_names = model.gene_names
print(f"Model expects {len(train_gene_names)} genes.")

# Manually load and align data for prediction
predict_id = test_ids[0]
print(f"Manually preparing prediction data for ID: {predict_id}")

handler = MilVisiumHandler()
embedding_path = Path(config.data.data_dir) / "cell_embeddings" / f"{config.data.embedding_tag}_{predict_id}_{config.data.shape_name}.h5"

# Load adata - this sets adata.uns["cell_embedding_path"] and adata.uns["spot_cell_map"]
adata = handler.load_and_preprocess_data(
    data_dir=Path(config.data.data_dir),
    id=predict_id,
    filter_genes=config.data.filter_genes,
    filter_cells=config.data.filter_cells,
    normalize=config.data.normalize,
    log1p=config.data.log1p,
    embedding_path=embedding_path,
    shape_name=config.data.shape_name,
)

# Convert sparse to dense if needed (same as STDataModule)
from scipy.sparse import issparse
if issparse(adata.X):
    adata.X = adata.X.toarray()

# --- Gene Alignment Logic ---
import pandas as pd
import anndata as ad
import scipy.sparse
import numpy as np

# CRITICAL: Backup ALL metadata before any transformation
# EmbeddedMilDataset requires both of these
saved_uns = adata.uns.copy()
saved_obsm = adata.obsm.copy() if hasattr(adata, 'obsm') and len(adata.obsm) > 0 else {}
saved_obs = adata.obs.copy()

# 1. Filter out genes not in training set (common genes)
common = sorted(list(set(adata.var_names).intersection(train_gene_names)))
adata = adata[:, common].copy()

# 2. Identify missing genes
missing_genes = sorted(list(set(train_gene_names) - set(adata.var_names)))

if len(missing_genes) > 0:
    print(f"Warning: Prediction set is missing {len(missing_genes)} genes. Filling with zeros.")
    
    # Prepare zero-filled matrix for missing genes
    zero_X = np.zeros((adata.n_obs, len(missing_genes)), dtype=adata.X.dtype)
    
    # Concatenate horizontally
    new_X = np.hstack([adata.X, zero_X])
    new_var_names = list(adata.var_names) + missing_genes
    
    # Create new AnnData with combined data
    adata = ad.AnnData(
        X=new_X,
        obs=saved_obs,  # Use saved obs
        var=pd.DataFrame(index=new_var_names),
    )

# CRITICAL: Restore metadata after transformation
adata.uns = saved_uns
if saved_obsm:
    adata.obsm = saved_obsm

# 3. Reorder strictly to match training genes order
# AND backup/restore uns again because slicing can drop it
saved_uns_final = adata.uns.copy()
adata = adata[:, train_gene_names].copy()
adata.uns = saved_uns_final

print(f"Aligned adata shape: {adata.shape}")
print(f"adata.uns keys: {list(adata.uns.keys())}")

# Create Dataset manually
# MilVisiumHandler.create_dataset expects adata with:
# - adata.uns["cell_embedding_path"]
# - adata.uns["spot_cell_map"] 
# - adata.obs["size_factor"]
predict_ds = handler.create_dataset(adata, embedding_path)

# Create DataLoader with custom_collate (required for MIL)
from torch.utils.data import DataLoader
from scellst.dataset.dataset_utils import custom_collate
from scellst.constant import REGISTRY_KEYS

loader = DataLoader(
    predict_ds, 
    batch_size=config.data.batch_size, 
    shuffle=False, 
    num_workers=0,
    collate_fn=custom_collate
)

# Prediction Loop
preds = []
truths = []

print(f"Predicting on {predict_id}...")
from tqdm import tqdm
import sys

with torch.no_grad():
    for batch in tqdm(loader, desc="Predicting", file=sys.stdout):
        batch_to_device = {}
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch_to_device[k] = v.to(model.device)
            else:
                batch_to_device[k] = v
        
        # Call internal model forward (model.model is InstanceDistributionMilModel)
        bag_dict, _ = model.model(batch_to_device)
        
        # CORRECT KEY: REGISTRY_KEYS.OUTPUT_PREDICTION = "output_prediction"
        pred = bag_dict[REGISTRY_KEYS.OUTPUT_PREDICTION].cpu().numpy()
        
        # CORRECT KEY: REGISTRY_KEYS.Y_BAG_KEY = "Y_bag"
        truth = batch[REGISTRY_KEYS.Y_BAG_KEY].cpu().numpy()
        
        preds.append(pred)
        truths.append(truth)

pred_bag = np.concatenate(preds, axis=0)
true_bag = np.concatenate(truths, axis=0)

print(f"Prediction shape: {pred_bag.shape}")
print(f"Truth shape: {true_bag.shape}")

# Save Results
np.save(RESULT_DIR / 'pred_bag.npy', pred_bag)
np.save(RESULT_DIR / 'true_bag.npy', true_bag)
print(f"Saved results to {RESULT_DIR}")

Loading model from D:\code\Morpho-VC\benchmark\results\sCellST\checkpoints\best_model.ckpt
19:20:59 | INFO | GeneLightningModel(
  (train_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef(),
    prefix=val/
  )
  (valid_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef(),
    prefix=val/
  )
  (test_metrics): MetricCollection(
    (mse): MeanSquaredError()
    (pcc): PearsonCorrCoef()
    (scc): SpearmanCorrCoef()
  )
  (model): InstanceMilModel(
    (gene_predictor): GenePredictor(
      (final_activation_layer): Softplus(beta=20, threshold=20.0)
      (model): Sequential(
        (0): Sequential(
          (0): Linear(in_features=2048, out_features=256, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
          (2): Dropout(p=0.1, inplace=False)
        )
        (1): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
          (2): 

## 4. Evaluation

Calculate MAE, RMSE, Pearson Correlation.

In [5]:
def pearson_corr(a, b):
    if np.all(a == a[0]) or np.all(b == b[0]):
        return np.nan
    a = a - a.mean()
    b = b - b.mean()
    denom = np.sqrt((a * a).sum()) * np.sqrt((b * b).sum())
    if denom == 0:
        return np.nan
    return float((a * b).sum() / denom)

mae = np.mean(np.abs(pred_bag - true_bag))
rmse = np.sqrt(np.mean((pred_bag - true_bag) ** 2))

# Gene-wise Pearson
gene_corrs = []
for i in range(pred_bag.shape[1]):
    corr = pearson_corr(pred_bag[:, i], true_bag[:, i])
    gene_corrs.append(corr)

valid = [(i, c) for i, c in enumerate(gene_corrs) if not np.isnan(c)]
mean_gene_corr = float(np.mean([c for _, c in valid])) if valid else float('nan')

best_gene = max(valid, key=lambda x: x[1]) if valid else (None, None)
best_gene_name = common_genes[best_gene[0]] if best_gene[0] is not None else "NA"

print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'Mean Pearson: {mean_gene_corr:.4f}')
print(f'Best Gene: {best_gene_name} ({best_gene[1]:.4f})')

metrics = {
    'MAE': float(mae),
    'RMSE': float(rmse),
    'Mean_Pearson': float(mean_gene_corr),
    'Best_Gene': best_gene_name,
    'Best_Pearson': float(best_gene[1]) if best_gene[1] is not None else None
}
with open(RESULT_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

MAE: 0.3363
RMSE: 0.4483
Mean Pearson: 0.0779
Best Gene: LMX1B (0.6885)
